In [1]:
from schnapsen.bots import RdeepBot
from schnapsen.bots
import random 

bot1 = RdeepBot(10, 10, random.Random(40), "Rdeepbot_1")
bot2 = RdeepBot(10, 10, random.Random(40), "RdeepBot_2")

create_replay_memory_dataset(bot1=bot1, bot2=bot2)


ImportError: attempted relative import with no known parent package